In [1]:
import requests
import os

In [2]:
# Get API key from environment variable
key = os.environ['CTPP_API_KEY']
header = {"x-api-key": key}

In [3]:
# Set base URL for API
server = "http://ctpp.macrosysrt.com/api"

In [4]:
# List all available datasets
endpoint = "/datasets"
response = requests.get(url=server+endpoint, headers=header)
print(response.text)

{"size":1,"page":1,"total":1,"data":[{"year":2016,"title":"2012-2016 (5-year)","description":"2016 dataset"}]}


In [6]:
# List all groups for year 2016 matching keyword "means of transportation"
endpoint = "/groups"
payload = {"year": 2016}
response = requests.get(server+endpoint, headers=header, params=payload)
# Uncomment the next line if you want to see what the actual response looks like
# print(response.text)
output = response.json()
for group in output['data']:
    name = group["name"]
    desc = group["description"]
    print(f"{name}: {desc}")

A101100 : Total Population (1)
A101101: Age (11)
A101102: Urban/Rural residence (3)
A101103: Hispanic origin (3)
A101104: Length of US Residence (6)
A101105: Minority status (3)
A101106: Unweighted sample count of the population (1)
A101108: Race (5)
A101109: School Enrollment (7)
A101110: Sex (3)
A101202: Age (9) by School enrollment (7)
A101204: Hispanic Origin (3) by Race of Person (5) 
A102101: Total workers (1)
A102102: Age of worker (8)
A102103: Class of worker (9)
A102104: Earnings in the past 12 months (2016$) (11)
A102105: Industry (15)
A102106: Means of transportation (18)
A102107: Occupation (25)
A102108: Time leaving home (41)
A102109: Usual Hours worked per week (7)
A102110: Travel time to work (12)
A102204: Earnings in the past 12 months (2016$) (11) by Travel time (12)
A102205: Hispanic Origin (3) by Race (5)
A102214: Occupation (25) by Industry (15)
A103100: Total Workers in households (1)
A103204: Household income in the past 12 months (2016$) (9) by Earnings in the pa

In [9]:
# list all variables for group A102106: Means of transportation (18)
endpoint = "/groups/A102106/variables"
payload = {"year": 2016}
response = requests.get(server+endpoint, headers=header, params=payload)
# Uncomment the next line if you want to see what the actual response looks like
# print(response.text)
output = response.json()
## Code below should not be necessary as of current version but shows how to resize request if needed
#print(f'{output["total"]} variables returned')
#if output["total"]>100: # check if there are more columns than the default page size
#    payload["size"] = output["total"] # resize the page to include all variables
#    response = requests.get(server+endpoint, headers=header, params=payload)
#    output = response.json()
colnames = [] # a list to hold column names (for later)
for column in output["data"]:
    name = column["name"]
    desc = column["label"]
    if "_e" in name: # only report estimates
        print(f"{name}: {desc}")
        vname, suffix = name.split("_")
        colnames.append(desc)
numcols = len(colnames)
print(f"{numcols} items selected")

A102106_e1: Total
A102106_e2: Car, truck, or van -- Drove alone
A102106_e3: Car, truck, or van -- In a 2-person carpool
A102106_e4: Car, truck, or van -- In a 3-person carpool
A102106_e5: Car, truck, or van -- In a 4-person carpool
A102106_e6: Car, truck, or van -- In a 5-or-6-person carpool
A102106_e7: Car, truck, or van -- In a 7-or-more-person carpool
A102106_e8: Bus or trolley bus
A102106_e9: Streetcar or trolley car
A102106_e10: Subway or elevated
A102106_e11: Railroad
A102106_e12: Ferryboat
A102106_e13: Bicycle
A102106_e14: Walked
A102106_e15: Taxicab
A102106_e16: Motorcycle
A102106_e17: Other method
A102106_e18: Worked at home
18 items selected


In [10]:
import pandas as pd # for table display
# get data with geography filter rutned on (Washington, DC)
endpoint = "/data/2016"
payload = {"in": 'state:11', "for": 'county:001', "get": 'group(A102106)'}
response = requests.get(server+endpoint, headers=header, params=payload)
# Uncomment the next line if you want to see what the actual response looks like
#print(response.text)
output = response.json()
location = output["data"][0]["name"]
geoid = output["data"][0]["geoid"]
estimates = numcols * [0]
MOEs = numcols * [0]
for variable in output["data"][0].keys():
    if variable in ["geoid","name"]:
        continue
    vname, suffix = variable.split("_")
    index = int(suffix[1:])-1
    if suffix[0] == "e":
        value = int(output["data"][0][variable].replace(",",""))
        # put the value in the estimates list
        estimates[index] = value
        if variable == "A102106_e1":
            total = value
    elif suffix[0] == "m":
        value = int(output["data"][0][variable][3:].replace(",",""))
        # extract numeric portion only and put it in the MOEs list
        MOEs[index] = value
df = pd.DataFrame(
    {
        "Variable": pd.Series(colnames),
        "MOE": pd.Series(MOEs, dtype="int32"),
        "Estimate": pd.Series(estimates, dtype="int32")
    }
)
df["Low"] = df.Estimate - df.MOE
df["High"] = df.Estimate + df.MOE
df["Est. %"] = round(100*df.Estimate / total,2)
df

,Variable,MOE,Estimate,Low,High,Est. %
0,Total,2228,345735,343507,347963,100.00
1,"Car, truck, or van -- Drove alone",2356,116520,114164,118876,33.70
2,"Car, truck, or van -- In a 2-person carpool",832,13885,13053,14717,4.02
3,"Car, truck, or van -- In a 3-person carpool",434,3060,2626,3494,0.89
4,"Car, truck, or van -- In a 4-person carpool",285,1195,910,1480,0.35
5,"Car, truck, or van -- In a 5-or-6-person carpool",200,475,275,675,0.14
6,"Car, truck, or van -- In a 7-or-more-person ca...",92,185,93,277,0.05
7,Bus or trolley bus,1770,52150,50380,53920,15.08
8,Streetcar or trolley car,284,980,696,1264,0.28
9,Subway or elevated,1659,72800,71141,74459,21.06
